## Extract All Rural & Urban Data
Author: Jennifer Grant


We need to check whether our model can perform well on both rural and urban data. This is important because the Continental United States is made up of urban and rural domains, both of which have completely different properties in their vertical profiles. We need our model to pick up on these differences when making predictions. 

We need to train our model on all available rural and urban data for Atlanta. This notebook extracts all data from rural and urban cells who have a 35km radius of surrounding information available. This excludes any cells on the outer edges, plus some. Note that 35km is the radius we are using for our model. 

####Import Libraries

In [ ]:
# Cannot live without our libraries
!pip install netCDF4
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import *

     |████████████████████████████████| 4.3MB 2.7MB/s 
     |████████████████████████████████| 296kB 28.1MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Import Data

In [ ]:
#first need to mount our drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


As mentioned we need to import both urban and rural data. We first start by importing our raw data for urban Atlanta. 

In [ ]:
#import urban data files
file_endings = ['05', '07', '08', '09', '11', '12', '13', '14']
data_path = 'drive/My Drive/Atlanta/raw_data/met_atlanta_full_20'
urban_objects = {}

for i in np.arange(len(file_endings)):
    urban_objects[i] = Dataset(data_path + file_endings[i])

Next we import our raw data for rural Atlanta.

In [ ]:
#import rural data files
data_path = 'drive/My Drive/BroaderAtlanta/raw_data/met_broader_atlanta_20'
rural_objects = {}

for i in np.arange(len(file_endings)):
    rural_objects[i] = Dataset(data_path + file_endings[i])

We are fortunate enough to have the $cos(zenithangle)$ in our rural Atlanta dataset, but we are not so fortunate for our urban Atlanta dataset. So lastly We must import the zenith angle data for urban Atlanta. 

In [ ]:
#import zenith angle data for our urban dataset (rural already has it)
data_path = 'drive/My Drive/Atlanta/zenith_angle_data/met_atlanta_zen_20'
zenith_objects = {}

for i in np.arange(len(file_endings)):
    zenith_objects[i] = Dataset(data_path + file_endings[i])

#### Grabbing Cell Indices

Now that we have our data we need to extract information for each cell represented in our data over the given years. We start this task by finding which cells from both rural & urban datasets have a radius of 35km surrounding information available. If a cell does not have this radius of available surrounding information, then the dataframe cannot be homogenous in dimensions/features as the others, and so we will not use these cells in our model. Note that we use the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) below in order to calculate distances that account for the curvature of the earth. 

In [ ]:
## We first build a couple of functions that will help us find surrounding
## indices for each cell.
##
## Function that returns indices of surrounding cells within a target radius away.
def haversine_distance(lons, lats, center_lon, center_lat):
  # more variables needed for the formula
  a = []
  distances = []
  earth_radius = 6371.009
  lat_diff = [radians(lat - center_lat) for lat in lats]
  lon_diff = [radians(lon - center_lon) for lon in lons]
  
  # need to convert lon,lat & center cell coordinates to radians for equation
  x_lat_rad = [radians(lat) for lat in lats]
  x_lon_rad = [radians(lon) for lon in lons]
  center_lat_rad = radians(center_lat)
  center_lon_rad = radians(center_lon)
  
  #calculate distances using Haversine formula
  for i in np.arange(len(lats)):
      a.append(sin(lat_diff[i]/2)**2 + cos(x_lat_rad[i])*cos(center_lat_rad)*sin(lon_diff[i]/2)**2)
      c = 2*atan2(sqrt(a[i]), sqrt(1 - a[i]))
      distances.append(earth_radius*c)
  
  return find_surrounding_indices(distances)

# grabs indices of cells within a target_distance away
def find_surrounding_indices(distances):
  target_distance = 35
  surrounding_indices = []

  for distance in distances:
    if ((distance <= target_distance) & (distance !=0)): #0 means its our center cell, so we exclude
        surrounding_indices.append(distances.index(distance))
  return surrounding_indices

In [ ]:
#grab the lon, lat data from rural datafiles
rural_lats = rural_objects[0].variables['xlat'][:][0].data
rural_lons = rural_objects[0].variables['xlon'][:][0].data

#grab the lon, lat data from urbal datafiles
urban_lats = urban_objects[0].variables['xlat'][:][0].data
urban_lons = urban_objects[0].variables['xlon'][:][0].data

# need number of cells in each dataset - lat/lon tells us this
num_rural_cells = len(rural_objects[0].variables['xlat'][0]) 
num_urban_cells = len(urban_objects[0].variables['xlat'][0])

#initialize dictionaries to hold a cell's surrounding indices
rural_surroundings = {}
urban_surroundings = {}

#initialize keys and empty list for each key which will be populated with 
# surrounding indices
for i in np.arange(num_rural_cells):
  rural_surroundings[i] = []

for i in np.arange(num_urban_cells):
  urban_surroundings[i] = []

#here we go
for i in np.arange(num_rural_cells):
  #grab coordinates of cell
  center_lat = rural_objects[0].variables['xlat'][:][0][i]
  center_lon = rural_objects[0].variables['xlon'][:][0][i]

  #call function
  rural_surroundings[i].extend(haversine_distance(rural_lons, rural_lats, center_lon, center_lat))

for i in np.arange(num_urban_cells):
  #grab coordinates of cell
  center_lat = urban_objects[0].variables['xlat'][:][0][i]
  center_lon = urban_objects[0].variables['xlon'][:][0][i]

  #call function
  urban_surroundings[i].extend(haversine_distance(urban_lons, urban_lats, center_lon, center_lat))

If you explore the surrounding indices of each cell in either the rural or urban dictionaries populated above, you'll see that they all don't have the same number of surrounding indices. We know that 35km of surrounding information should be 24 cells, so we will filter out any cells that do not have have 24 surrounding indices in the dictionaries.

In [ ]:
#initialize constant for number of surrounding indices needed
num_surroundings = 24

#first filter rural dictionary
for i in np.arange(num_rural_cells):
  if len(rural_surroundings[i]) != num_surroundings:
    del rural_surroundings[i]

#next is urban dictionary
for i in np.arange(num_urban_cells):
  if len(urban_surroundings[i]) != num_surroundings:
    del urban_surroundings[i]

Fantastic! Last thing we need to adjust is addressing the fact that there are some cells that overlap between our rural and urban datasets. We check to see if any overalapping cells survived the filtering process above and are present in both the rural and urban dictionaries. If so, we keep only one copy - which dictionary we keep it in doesn't matter because there is no feature in our model indicating urban or rural. So long as we include it somewhere.

In [ ]:
#lets grab the filtered keys in order to get their coordinates and compare
rural_keys = list(rural_surroundings.keys())
urban_keys = list(urban_surroundings.keys())

#lets put (lon, lats) together as a tuple - easier for comparison
rural_coordinates = [(rural_lons[key], rural_lats[key]) for key in rural_keys]
urban_coordinates = [(urban_lons[key], urban_lats[key]) for key in urban_keys]

#lets check to see if there were actually any overalpping cells removed by 
# checking length of dictionary before and after
old_length = len(urban_surroundings)

#now lets compare tuples to find our overlapping cells and remove them from the 
# urban dictionary
for coordinate in urban_coordinates:
  if (coordinate in rural_coordinates):
    key_of_coordinate = urban_keys[urban_coordinates.index(coordinate)]
    del urban_surroundings[key_of_coordinate]

#grab new length after removing any potential overlapping cells
new_length = len(urban_surroundings)

print('Old length: {}\nNew length: {}'.format(old_length, new_length))

Old length: 17
New length: 9


In [ ]:
rural_surroundings.keys()

dict_keys([25, 26, 27, 28, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 109, 110, 111, 112, 113, 114])

Looks like we did have some overlapping cells afterall. We move onto building dataframes for keys (cells) in both dictionaries.

#### Building Dataframes

We move onto building dataframes for the surviving cells. Note that the rural dataset already has the cosine zenith angle feature in it, while the urban dataset does not. For this reason we need to handle building their respective dataframes differently. 

In [ ]:
# functions that we will use to help build the dictionary of data to feed into a dataframe object

# initialize the number of iterations needed to run loops/list comprehension - 
# this dimension is common to both rural & urban dataset
num_arrays = rural_objects[0].variables['no2'].shape[0] #could have picked any variable with first dimension=744 for either rural or urban (same for both)
    
def extract_rural_data(label, file, cell_index):
  print(label)
  data = []
  if (label == 'W' or label == 'elev'):
      #skip last element (top most layer) for these features to make even 29 layers
      for i in np.arange(num_arrays):
        data.extend(file.variables[label][i][cell_index].data[:29])
  elif (label == 'zenith_angle'):
    for i in np.arange(num_arrays):
      #take arc cos to get the actual zenith angle
      data.append(acos(file.variables['COSZEN'][i][cell_index].data) * 180 / pi)
  else:
    #extract all layers for every other feature
    for i in np.arange(num_arrays):
      data.extend(file.variables[label][i][cell_index].data)
  return data


def extract_urban_data(label, file, zenith_file, cell_index):
  print(label)
  data = []
  if (label == 'W' or label == 'elev'):
      #skip last element (top most layer) for these features to make even 29 layers
      for i in np.arange(num_arrays):
        data.extend(file.variables[label][i][cell_index].data[:29])
  elif (label == 'zenith_angle'):
    for i in np.arange(num_arrays):
      #take arc cos to get the actual zenith angle
      data.append(acos(zenith_file.variables['COSZEN'][i][cell_index].data) * 180 / pi)
  else:
    #extract all layers for every other feature
    for i in np.arange(num_arrays):
      data.extend(file.variables[label][i][cell_index].data)
  return data

In [ ]:
#this function builds a rural dataframe for any cell_index given
def build_rural_frame(cell_index):

  #build list of feature labels
  var_labels = []
  for variable in rural_objects[0].variables:    
      if (variable == 'xlon' or variable == 'xlat' or variable == 'date' or variable == 'hour'):
          continue
      elif (variable == 'COSZEN'):
          var_labels.append('zenith_angle')
      else:
          var_labels.append(variable)

  #build dictionary that will hold the data
  data_dict = {label: [] for label in var_labels} #assign labels so that you can just extend the value with the returned array

  for i in np.arange(len(rural_objects)):
    #we print the file number and which feature the alogorithm is extracting
    #data for
    print('\nFile {}\n'.format(i + 1))
    for variable in var_labels:
        data_dict[variable].extend(extract_rural_data(variable, rural_objects[i], cell_index))
    
  # We need to adjust the cumulative flashcounts to the actual flashcounts
  num_months = len(rural_objects) #how many months of data we have
  num_per_month = 744 #number of flashcounts per month
  updated_ic_flash = []
  updated_cg_flash = []
  ic_flash = data_dict['IC_FLASHCOUNT']
  cg_flash = data_dict['CG_FLASHCOUNT']

  for i in np.arange(num_months):
    #grab each month separately
    month_ic_flashes = ic_flash[i*744:i*744+744]
    month_cg_flashes = cg_flash[i*744:i*744+744]

    #calculate difference
    ic_diff = np.diff(month_ic_flashes)
    cg_diff = np.diff(month_cg_flashes)
    updated_ic_flash.extend(ic_diff)
    updated_cg_flash.extend(cg_diff)

    #since we don't have continuous data we append one last zero
    updated_ic_flash.append(0)
    updated_cg_flash.append(0)

  #need to repeat each entry of pblh, ic_flashcount, cg_flashcount, date, hour for
  #each vertical layer
  data_dict['PBLH'] = np.repeat(data_dict['PBLH'], 29)
  data_dict['IC_FLASHCOUNT'] = np.repeat(updated_ic_flash, 29)
  data_dict['CG_FLASHCOUNT'] = np.repeat(updated_cg_flash, 29)

  # recall that E_NO only had 19 layers and we want to make it 29 to match the
  # dimensions of the other features. We fix this by adding an additional ten
  # zeros to the end of each 19 layers

  zeros = np.zeros(10)
  num_obs = int(len(data_dict['E_NO']) / 19)   #how many groups of 19 E_NO we have
  e_no = []

  for i in np.arange(num_obs):
    one_profile = data_dict['E_NO'][i*19:i*19+19]
    one_profile.extend(zeros)
    e_no.extend(one_profile)

  #replace old list in dictionary with updated one
  data_dict['E_NO'] = e_no

  #extract date and hour, then repeat its entries as well
  hour = []
  date = []

  for i in np.arange(len(rural_objects)):
    #flattening the lists so its one list containing all elements
    date.extend([item for sublist in rural_objects[i].variables['date'][:] 
                for item in sublist])
    hour.extend([item for sublist in rural_objects[i].variables['hour'][:] 
                for item in sublist])

  dates = np.repeat(date, 29)
  hours = np.repeat(hour, 29)

  #also need to repeat zenith angle for all 29 layers 
  data_dict['zenith_angle'] = np.repeat(data_dict['zenith_angle'], 29)

  #we add the date and hour lists to the dictionary
  data_dict['date'] = dates
  data_dict['hour'] = hours
  
  cell = pd.DataFrame(data_dict)
  #putting the date and hour first
  cell = cell[['date', 'hour', 'no2', 'U', 'V', 'W', 'PBLH', 'E_NO',
                            'IC_FLASHCOUNT', 'CG_FLASHCOUNT', 'pres', 'elev',
                            'temp', 'zenith_angle']]
  
  file_label = 'rural_atlanta_' + str(cell_index) + '.csv'
  cell.to_csv('drive/My Drive/urban_and_rural/rural/individual_data/' + file_label)

In [ ]:
#this function builds an urban dataframe for any cell_index given
def build_urban_frame(cell_index):

  #build list of feature labels
  var_labels = []
  for variable in urban_objects[0].variables:    
      if (variable == 'xlon' or variable == 'xlat' or variable == 'date' or variable == 'hour'):
          continue
      else:
          var_labels.append(variable)
  
  #don't have zenith angle in urban dataset so need to append it separately
  var_labels.append('zenith_angle')

  #build dictionary that will hold the data
  data_dict = {label: [] for label in var_labels} #assign labels so that you can just extend the value with the returned array

  for i in np.arange(len(urban_objects)):
    #we print the file number and which feature the alogorithm is extracting
    #data for
    print('\nFile {}\n'.format(i + 1))
    for variable in var_labels:
        data_dict[variable].extend(extract_urban_data(variable, urban_objects[i], zenith_objects[i], cell_index))
    
  # We need to adjust the cumulative flashcounts to the actual flashcounts
  num_months = len(urban_objects) #how many months of data we have
  num_per_month = 744 #number of flashcounts per month
  updated_ic_flash = []
  updated_cg_flash = []
  ic_flash = data_dict['IC_FLASHCOUNT']
  cg_flash = data_dict['CG_FLASHCOUNT']

  for i in np.arange(num_months):
    #grab each month separately
    month_ic_flashes = ic_flash[i*744:i*744+744]
    month_cg_flashes = cg_flash[i*744:i*744+744]

    #calculate difference
    ic_diff = np.diff(month_ic_flashes)
    cg_diff = np.diff(month_cg_flashes)
    updated_ic_flash.extend(ic_diff)
    updated_cg_flash.extend(cg_diff)

    #since we don't have continuous data we append one last zero
    updated_ic_flash.append(0)
    updated_cg_flash.append(0)

  #need to repeat each entry of pblh, ic_flashcount, cg_flashcount, date, hour for
  #each vertical layer
  data_dict['PBLH'] = np.repeat(data_dict['PBLH'], 29)
  data_dict['IC_FLASHCOUNT'] = np.repeat(updated_ic_flash, 29)
  data_dict['CG_FLASHCOUNT'] = np.repeat(updated_cg_flash, 29)

  # recall that E_NO only had 19 layers and we want to make it 29 to match the
  # dimensions of the other features. We fix this by adding an additional ten
  # zeros to the end of each 19 layers

  zeros = np.zeros(10)
  num_obs = int(len(data_dict['E_NO']) / 19)   #how many groups of 19 E_NO we have
  e_no = []

  for i in np.arange(num_obs):
    one_profile = data_dict['E_NO'][i*19:i*19+19]
    one_profile.extend(zeros)
    e_no.extend(one_profile)

  #replace old list in dictionary with updated one
  data_dict['E_NO'] = e_no

  #extract date and hour, then repeat its entries as well
  hour = []
  date = []

  for i in np.arange(len(urban_objects)):
    #flattening the lists so its one list containing all elements
    date.extend([item for sublist in urban_objects[i].variables['date'][:] 
                for item in sublist])
    hour.extend([item for sublist in urban_objects[i].variables['hour'][:] 
                for item in sublist])

  dates = np.repeat(date, 29)
  hours = np.repeat(hour, 29)

  #also need to repeat zenith angle for all 29 layers 
  data_dict['zenith_angle'] = np.repeat(data_dict['zenith_angle'], 29)

  #we add the date and hour lists to the dictionary
  data_dict['date'] = dates
  data_dict['hour'] = hours
  
  cell = pd.DataFrame(data_dict)
  #putting the date and hour first
  cell = cell[['date', 'hour', 'no2', 'U', 'V', 'W', 'PBLH', 'E_NO',
                            'IC_FLASHCOUNT', 'CG_FLASHCOUNT', 'pres', 'elev',
                            'temp', 'zenith_angle']]
  
  file_label = 'urban_atlanta_' + str(cell_index) + '.csv'
  cell.to_csv('drive/My Drive/urban_and_rural/urban/individual_data/' + file_label)

In [ ]:
#lets now call these functions to build our frames
for key in rural_keys:
  build_rural_frame(key)

In [ ]:
urban_keys = list(urban_surroundings.keys())

for key in urban_keys:
  build_urban_frame(key) 


File 1

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 2

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 3

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 4

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 5

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 6

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 7

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 8

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 1

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 2

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 3

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
elev
temp
zenith_angle

File 4

IC_FLASHCOUNT
CG_FLASHCOUNT
no2
U
V
W
PBLH
E_NO
pres
ele